In [2]:
from Bio import SeqIO
import pandas as pd
from scripts import utils
from Bio import Entrez
from more_itertools import chunked
from tqdm import tqdm
import pickle
import io

Entrez.email = "gernel@informatik.uni-freiburg.de"

In [3]:
rna_inter_df = utils.load_rna_inter_protein('UniProt')
rna_inter_df.to_parquet('RNAInter_uniport.parquet', engine='pyarrow', compression=None)

In [4]:
rna_inter_df = pd.read_parquet('RNAInter_uniport.parquet', engine='pyarrow')

In [6]:
uniprot_ids = list(rna_inter_df['Raw_ID2'].unique())
print(f"{len(uniprot_ids)} unique protein IDs")
uniprot_ids = [uniprot_id[8:] for uniprot_id in uniprot_ids]
protein_sequences = utils.fetch_protein_fasta(uniprot_ids)
# convert fasta string to Seq Objects
protein_sequences = [(protein[0], list(SeqIO.parse(io.StringIO(protein[1]), "fasta"))) for protein in protein_sequences]
# flatten list
protein_sequences = [(protein[0], seq) for protein in protein_sequences for seq in protein[1]]
uniprot_proteins_2_df = [dict(
    Raw_ID2=f"UniProt:{protein[0]}",
    Sequence_2=str(protein[1].seq),
    Sequence_2_len=len(str(protein[1].seq)),
    Sequence_2_ID=protein[1].id,
) for protein in protein_sequences]
uniprot_proteins_df = pd.DataFrame(uniprot_proteins_2_df)
uniprot_proteins_df.to_parquet("protein_uniprot.parquet", engine='pyarrow', compression=None)

3290 unique protein IDs
100 proteins done
200 proteins done
300 proteins done
400 proteins done
500 proteins done
600 proteins done
700 proteins done
800 proteins done
900 proteins done
1000 proteins done
1100 proteins done
1200 proteins done
1300 proteins done
1400 proteins done
1500 proteins done
1600 proteins done
1700 proteins done
1800 proteins done
1900 proteins done
2000 proteins done
2100 proteins done
2200 proteins done
2300 proteins done
2400 proteins done
2500 proteins done
2600 proteins done
2700 proteins done
2800 proteins done
2900 proteins done
3000 proteins done
3100 proteins done
3200 proteins done


In [8]:
uniprot_proteins_df = pd.read_parquet("protein_uniprot.parquet", engine='pyarrow')

# Calc recovery rate

In [9]:
utils.calc_recovery_rate(rna_inter_df, uniprot_proteins_df, col_name='Raw_ID2')

Unique Gene IDs before extraction:	3,290
Unique Gene IDs after extraction:	3,069
Extraction rate:	93.28%
